In [2]:
import sys
sys.path.append('../../')

In [8]:
# importnb allows us to load the code from other notebooks
from importnb import Notebook
with Notebook():
    import OfficeAutomation.common as common  # will probably become OfficeAutomation.common
    import OfficeAutomation.lamps as lamps

In [4]:
import crestdsl.model as crest
import crestdsl.simulation as crestsim
import crestdsl.ui as ui

In [9]:
class LampTest(common.TestClass):
    def test_switch_sets_req_electricity(self):
        need_watt = 12345
        lamp = lamps.Lamp()
        lamp.need_watt.value = need_watt
        lamp.electricity_in.value = 120
        
        sim = crestsim.Simulator(lamp)
        
        "(switch_in, state, expected_req)"
        params = [
            ("off", lamp.off, 0),
            ("on", lamp.off, need_watt),
            ("off", lamp.on, 0),
            ("on", lamp.on, need_watt)
        ]
        
        for switch_value, lampstate, expected_req in params:
            with self.subTest(switch_value=switch_value, lampstate=lampstate._name, expected_req=expected_req):
                lamp.switch_in.value = switch_value
                lamp.current = lampstate
                sim.stabilise()
                self.assertEqual(lamp.req_electricity_out.value, expected_req, f"Req_electricity value:'{lamp.req_electricity_out}', expected value '{expected_req}'")
                        
    def test_turn_on_to_on(self):
        lamp = lamps.Lamp()
        lamp.electricity_in.value = lamp.need_watt.value + 100        
        lamp.switch_in.value = "off"
        
        # gotta stabilise, then manually trigger the right transition, since we're non-deterministic
        sim = crestsim.Simulator(lamp)
        sim.stabilise()

        lamp.switch_in.value = "on"
        self.assertTrue(sim.is_enabled(lamp.turn_on), "Transition turn_on is enabled")
        
        sim.stabilise()

        self.assertEqual(lamp.current, lamp.on, "Current state is not on")
        self.assertEqual(lamp.req_electricity_out.value, lamp.need_watt.value) # electricity is drawn
    
    def test_off_draws_no_electricity(self):
        lamp = lamps.Lamp()
        lamp.electricity_in.value = lamp.need_watt.value + 100        
        lamp.switch_in.value = "off"
        
        sim = crestsim.Simulator(lamp)
        sim.stabilise()
        
        self.assertEqual(lamp.current, lamp.off)
        self.assertEqual(lamp.req_electricity_out.value, 0) # no electricity is drawn  

# LampTest.runall()

...
----------------------------------------------------------------------
Ran 3 tests in 0.054s

OK


In [10]:
class FragileLampTest(common.TestClass):
    
    def test_switch_sets_req_electricity(self):
        need_watt = 12345
        lamp = lamps.FragileLamp()
        lamp.need_watt.value = need_watt
        lamp.electricity_in.value = 120
        
        sim = crestsim.Simulator(lamp)
        
        "(switch, state, expected_req)"
        params = [
            ("off", lamp.off, 0),
            ("on", lamp.off, need_watt),
            ("off", lamp.on, 0),
            ("on", lamp.on, need_watt),
            ("off", lamp.spike, 1000),
            ("on", lamp.spike, 1000),
            ("off", lamp.broken, 0),
            ("on", lamp.broken, 0),
        ]
        
        for switch_value, lampstate, expected_req in params:
            with self.subTest(switch_value=switch_value, lampstate=lampstate._name, expected_req=expected_req):
                lamp.switch_in.value = switch_value
                lamp.current = lampstate
                sim.stabilise()
                self.assertEqual(lamp.req_electricity_out.value, expected_req, f"Req_electricity value:'{lamp.req_electricity_out}', expected value '{expected_req}'")
                        
    def test_turn_on_to_on(self):
        lamp = lamps.FragileLamp()
        lamp.electricity_in.value = lamp.need_watt.value + 100        
        lamp.switch_in.value = "off"
        
        # gotta stabilise, then manually trigger the right transition, since we're non-deterministic
        sim = crestsim.Simulator(lamp)
        sim.stabilise()

        lamp.switch_in.value = "on"
        self.assertTrue(sim.is_enabled(lamp.turn_on), "Transition turn_on is enabled")
        
        # fire specific transition
        sim.fire(lamp.turn_on)
        sim.stabilise()

        self.assertEqual(lamp.current, lamp.on, "Current state is not on")
        self.assertEqual(lamp.req_electricity_out.value, lamp.need_watt.value) # electricity is drawn
    
    def test_spike(self):
        lamp = lamps.FragileLamp()
        lamp.electricity_in.value = lamp.need_watt.value + 100        
        lamp.switch_in.value = "off"
        
        # gotta stabilise, then manually trigger the right transition, since we're non-deterministic
        sim = crestsim.Simulator(lamp)
        sim.stabilise()
        
        lamp.switch_in.value = "on"
        self.assertTrue(sim.is_enabled(lamp.to_spike), "Transition to_spike is enabled")
        
        # fire specific transition
        sim.fire(lamp.to_spike)
        sim.stabilise()
        
        self.assertEqual(lamp.current, lamp.spike)
        self.assertEqual(lamp.req_electricity_out.value, 1000) # spike electricity is drawn
        
        sim.advance(1)
        self.assertEqual(lamp.current, lamp.broken)
        self.assertEqual(lamp.req_electricity_out.value, 0) # spike electricity is drawn
    
    
    def test_off_draws_no_electricity(self):
        lamp = lamps.FragileLamp()
        lamp.electricity_in.value = lamp.need_watt.value + 100        
        lamp.switch_in.value = "off"
        
        sim = crestsim.Simulator(lamp)
        sim.stabilise()
        
        self.assertEqual(lamp.current, lamp.off)
        self.assertEqual(lamp.req_electricity_out.value, 0) # no electricity is drawn  
        
# FragileLampTest.runall()

....
----------------------------------------------------------------------
Ran 4 tests in 0.098s

OK


In [13]:
class EmergencyLampTest(common.TestClass):
        
    def test_turn_on_to_on(self):
        lamp = lamps.EmergencyLamp()
        lamp.electricity_in.value = 200 
        lamp.current = lamp.off
        
        # gotta stabilise, then manually trigger the right transition, since we're non-deterministic
        sim = crestsim.Simulator(lamp)

        self.assertTrue(sim.is_enabled(lamp.turn_on), "Transition turn_on is enabled")
        
        # fire specific transition
        sim.fire(lamp.turn_on)
        sim.stabilise()

        self.assertEqual(lamp.current, lamp.on, "Current state is not on")
        self.assertEqual(lamp.req_electricity_out.value, lamp.need_watt.value) # electricity is drawn
    
    def test_spike(self):
        lamp = lamps.EmergencyLamp()
        lamp.electricity_in.value = 200
        lamp.current = lamp.off

        # gotta stabilise, then manually trigger the right transition, since we're non-deterministic
        sim = crestsim.Simulator(lamp)
        
        self.assertTrue(sim.is_enabled(lamp.to_spike), "Transition to_spike is enabled")
        
        # fire specific transition
        sim.fire(lamp.to_spike)
        sim.stabilise()
        
        self.assertEqual(lamp.current, lamp.spike)
        self.assertEqual(lamp.req_electricity_out.value, 1000) # spike electricity is drawn
        
        sim.advance(1)
        self.assertEqual(lamp.current, lamp.broken)
        self.assertEqual(lamp.req_electricity_out.value, 0) # spike electricity is drawn
    
    
    def test_off_draws_no_electricity(self):
        lamp = lamps.EmergencyLamp()
        lamp.electricity_in.value = 0  # no electricity means it'll switch off
        lamp.current = lamp.on
        
        sim = crestsim.Simulator(lamp)
        sim.stabilise()
        
        self.assertEqual(lamp.current, lamp.off)
        self.assertEqual(lamp.req_electricity_out.value, 0) # no electricity is drawn  
        
# EmergencyLampTest.runall()